# Datasets

Run this `gcsfuse` cell if you can't list the folders inside of "/gcs"

In [5]:
!gcsfuse --implicit-dirs "~/gcs"

I1112 04:24:34.893978 2023/11/12 04:24:34.893952 Start gcsfuse/0.42.5 (Go version go1.20.3) for app "" using mount point: /home/jupyter/gcs


When using GCS buckets, use "/gcs" instead of "gs://"

In [6]:
#from sklearn.model_selection import train_test_split

dataset_path = "/home/jupyter/gcs/serena-shsw-datasets"

training_dataset = dataset_path + "/FER-2013/train"
test_dataset = dataset_path + "/FER-2013/test"
validation_dataset = dataset_path + "/FER-2013/valid"

# Split the dataset into training and validation sets
#training_dataset, validation_dataset = train_test_split(training_dataset, test_size=0.2, random_state=42)

# Output directory contents
!echo "Train"
!ls {training_dataset}
!echo "Test"
!ls {test_dataset}
!echo "Valid"
!ls {validation_dataset}

Train
angry  disgust	fear  happy  neutral  sad  surprise
Test
angry  disgust	fear  happy  neutral  sad  surprise
Valid


# Import Library

In [3]:
import csv
import string
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

2023-11-11 13:15:47.126791: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-11 13:15:48.579045: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-11 13:15:48.579089: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-11 13:15:52.194063: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

# Evaluate Model

In [4]:
saved_model_path = dataset_path + "/models/serena-emotion-detector/model"
model = tf.keras.models.load_model(saved_model_path, compile=True)

# Check its architecture
model.summary()

2023-11-11 13:16:01.135079: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-11 13:16:01.135732: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-11 13:16:01.135763: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (emotion-detector-nb): /proc/driver/nvidia/version does not exist
2023-11-11 13:16:01.136681: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 14,847,815
Trainable params: 133,127
Non-trainable params: 14,714,688
______________________________________

Check accuracy of model

In [5]:
training_labels = [] 
test_labels = [] 

val_loss, val_accuracy = model.evaluate(training_dataset, training_labels)
print(f'Validation Accuracy: {val_accuracy}')

test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f'Test Accuracy: {test_accuracy}')

ValueError: Failed to find data adapter that can handle input: <class 'str'>, (<class 'list'> containing values of types set())